In [3]:
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.tensorboard import SummaryWriter
import Model.mymodel as mymodel

In [4]:
set_frame = 50

epochs = 60
batch_sz = 10
checkpoint_frequency = 3
learning_rate = 0.00005
gamma = 0.5


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dl, val_dl, test_dl = mymodel.load_data()

dataloaders = {
    'train' : train_dl,
    'val' : val_dl,
    'test' : test_dl
}



network = mymodel.get_simple_conv_net()
network = network.to(DEVICE)

optim = SGD(network.parameters(), lr=learning_rate)
lr_sch = ExponentialLR(optim, gamma)

writer = SummaryWriter()
stopp = mymodel.EarlyStopper(patience = 5, tolerance = 2)

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [ ]:

mymodel.train_model(network,epochs,dataloaders,optim,lr_sch,writer,stopp,checkpoint_frequency)